In [1]:
import sys
sys.path.append('..')
import processing
import configs.common as cc

loader = processing.DatasetLoader('E:\\GitHub\\dataset\\np_dataset')
train_dataloader, test_dataloader = loader.get_dataloaders()
# random_sample = loader.get_random_sample('train')
# random_sample
for src, trg, meta in train_dataloader:
    break
# src, trg, meta = random_sample

In [2]:
from train import new_model
import torch
type = 'transformer'
model_name = 'loss_0.10_time_2025-04-23-08-40-46.pth'
model = new_model(type)
model.load_state_dict(torch.load(f'../pretrained/{type}/{model_name}'))
model.to('cuda')

Transformer(
  (token_embedding_table): Embedding(963, 256)
  (metadata_embedding_table): Embedding(154, 256)
  (positional_encoding): PositionalEncoding()
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Head(
            (key): Linear(in_features=256, out_features=32, bias=False)
            (query): Linear(in_features=256, out_features=32, bias=False)
            (value): Linear(in_features=256, out_features=32, bias=False)
            (dropout): Dropout(p=0.01, inplace=False)
          )
        )
        (proj): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.01, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=256, out_features=1024, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1024, out_features=256, bias=True)
          (3): Dropout(p=0.01, inplace=False)
        )
      )
      (ln1): Lay

In [3]:
def generate(model, context_len, token_ids, meta_ids, num_tokens=1000, device='cpu'):
    model.eval()
    generated = token_ids.detach().cpu().numpy().tolist()[0]

    with torch.no_grad():
            for _ in range(num_tokens):
                print(_, end="\r")
                logits = model(token_ids, meta_ids)  # (1, seq_len, vocab_size)
                logits_last = logits[:, -1, :]       # (1, vocab_size)
                probs = torch.softmax(logits_last, dim=-1)  # (1, vocab_size)

                # Get top-5
                topk_probs, topk_indices = torch.topk(probs, 2, dim=-1)  # Each: shape (1, 5)

                # Normalize to sum to 1
                topk_probs = topk_probs / topk_probs.sum(dim=-1, keepdim=True)

                # Sample
                next_token_idx = torch.multinomial(topk_probs, num_samples=1)  # (1, 1), value in [0..4]
                next_token = topk_indices.gather(1, next_token_idx)

                generated.append(next_token.item())

                token_ids = torch.cat([token_ids, next_token], dim=1)
                token_ids = token_ids[:, -context_len:]

            # If you want metadata, sample or set as zeros:
            # e.g., meta_ids = torch.cat([meta_ids, new_meta], dim=1)[:, -context_len:]

    return generated

new_seq = generate(model, cc.config.values.block_len, src[0].unsqueeze(0), meta[0].unsqueeze(0), num_tokens=1000, device='cuda')

In [4]:
decoded_notes_old = processing.decode(src[0])
processing.note_to_midi(decoded_notes_old, "comparison.mid")

decoded_notes_new = processing.decode(new_seq)
processing.note_to_midi(decoded_notes_new, "generated.mid")

In [5]:
model(src[0].unsqueeze(0), meta[0].unsqueeze(0)).argmax(-1)

tensor([[224, 261, 357, 861, 486,  62, 215, 263, 861, 481,  62, 215, 279, 364,
         861, 486,  66, 224, 276, 358, 861, 486,  62, 224, 273, 357, 861, 486,
          71, 224, 270, 861, 511,  62, 175, 329, 375, 861, 492,  31, 214, 329,
         357, 861, 475,  79, 187, 329, 861, 511,  67, 175, 329, 861, 481,  67,
         228, 278, 361, 861, 486,  67, 224, 263, 370, 861, 486,  62, 224, 261,
         357, 861, 486,  59, 224, 258, 861, 486,  62, 224, 267, 366, 861, 486,
          59, 224, 267, 357, 861, 486,  67, 224, 267, 861, 481,  62, 224, 282,
         861, 486,  62, 224, 264, 375, 861, 486,  59, 224, 275, 357, 861, 486,
          67, 224, 263, 861, 481,  64, 229, 278, 364, 861, 486,  62, 223, 267,
         358, 861, 486,  67, 224, 266, 357, 861, 475,  73, 187, 329, 375, 861,
         511,  61, 175, 329, 357, 861, 492,  33, 214, 329, 861, 486,  69, 224,
         329, 861, 486,  64, 175, 329, 861, 486,  64, 224, 270, 861, 486,  64,
         224, 270, 861, 475,  81, 187, 329, 861, 475